In [15]:
%reload_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import path
import matplotlib.animation as animation
import matplotlib
import time

import glob
import json
from scipy.signal import find_peaks, butter, filtfilt
from model.bayes_filter import ParticleFilter
from sites.config import *
from util.general_util import *
from util.data_util import *
from model import *

In [2]:
matplotlib.rcParams['font.size'] = 16

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [8]:
def read_reference_points(grid_path):
    rp_locs, rp_features = [], []
    with open(grid_path) as f:
        lines = f.readlines()
    for line in lines:
        x, y, *features = line.split(' ')
        features = list(map(float, features))
        rp_locs.append((int(x),int(y)))
        rp_features.append(features)

    return rp_locs, rp_features

def smooth(seq, interval=200):
    #seq = [(v1,ts),...]
    smoothed = []
    for x in seq:
        sub_array = [s[0] for s in seq if x[1] <= s[1] <= x[1] + interval]
        smoothed.append(np.mean(sub_array, axis=0).tolist())
    return smoothed
    
def readInsData(directory):
    files = sorted(glob.glob(os.path.join(directory, 'na*_Sensors.txt')))
    paths = []
    for file in files:
        path = []
        lines = open(file).readlines()
        for line in lines[3:]:
            line = json.loads(line)
            if line['type'] in {1,2,3,4,9}:
                path.append(line)
        paths.append(path)
    return paths

def readData(directory, window_size = 1000):
    sensor_lookup = {1:0, 2:1, 3:2, 4:3, 9:4}
    wifi_files = sorted(glob.glob(os.path.join(directory, 'na*_WiFi.txt')))
    sensor_files = sorted(glob.glob(os.path.join(directory, 'na*_Sensors.txt')))
    google_files = sorted(glob.glob(os.path.join(directory, 'na*_Google.txt')))
    gps_files = sorted(glob.glob(os.path.join(directory, 'na*_GPS.txt')))

    paths = []
    for w,s,g,gg in zip(wifi_files, sensor_files, google_files, gps_files):
        print(w)
        path = []
        lines = open(w).readlines()       
        for i in range(3,len(lines)):
            ts, readings = lines[i].split(maxsplit=1)
            lines[i] = "%s wifi %s" % (ts, readings)
                       
        lines2 = []
        for line in open(s).readlines()[3:]:
            line = json.loads(line)
            if line["type"] in {1,2,3,4,9}:
                lines2.append("%d sensor %d %s" % (line["timestamp"], line["type"], ','.join(map(str, line["values"]))))
        
        lines3 = []
        for line in open(g).readlines()[1:]:
            line = json.loads(line)
            lines3.append("%d google %f,%f,%f" % (line["timestamp"], line["latitude"], line["longitude"], line["accuracy"]))
        
        lines4 = []
        for line in open(gg).readlines()[1:]:
            line = json.loads(line)
            lines4.append("%d gps %f,%f,%f" % (line["timestamp"], line["latitude"], line["longitude"], line["accuracy"]))
        
        points = lines[1].strip().split()
        points = [tuple(map(float, p.split(','))) for p in points]
        timestamps = list(map(int, lines[2].strip().split()))
        lines = list(sorted(lines[3:] + lines2 + lines3 + lines4))
        del lines2, lines3, lines4
                
        i = -1
        last_ts = timestamps[0]
        sensor_tmp = [[], [], [], [], []]
        google_tmp = None
        gps_tmp = None
        wifi_tmp = None
        for idx in range(len(lines)):
            ts, stype, *tmp = lines[idx].split()
            ts = int(ts)
            
            if ts > timestamps[i+1]:
                i += 1
                        
            if stype == 'sensor':
                if tmp[0] in {'1','2','3','4','9'}:
                    sensor_tmp[sensor_lookup[int(tmp[0])]].append((list(map(float, tmp[1].split(',')))  , ts))
            elif stype == 'google':
                google_tmp = list(map(float, tmp[0].split(',')))
            elif stype == 'wifi':
                if wifi_tmp is None or len(tmp) > len(wifi_tmp):
                    wifi_tmp = tmp
            elif stype == 'gps':
                gps_tmp = list(map(float, tmp[0].split(',')))
                
            if ts - last_ts >= window_size:
                ## pack all data
                ratio = (ts - timestamps[i]) / (timestamps[i+1] - timestamps[i])
                loc = (points[i][0]+ratio*(points[i+1][0]-points[i][0]), points[i][1]+ratio*(points[i+1][1]-points[i][1]))
                
                d = {}
                d['timestamp'] = ts
                d['loc'] = loc
                
                if gps_tmp is not None:
                    d['gps'] = gps_tmp
                
                if google_tmp is not None:
                    d['google'] = google_tmp
                    
                if wifi_tmp is not None:
                    ap_list, rssi_vec = [], []
                    for t in wifi_tmp:
                        mac, rssi = t.split(',')
                        ap_list.append(mac.lower())
                        rssi_vec.append(float(rssi))
                    if max(rssi_vec) > -70:
                        d['wifi'] = (np.array([rssi_vec]), ap_list)
                
                for j, k in enumerate(['acc', 'mag', 'heading', 'gyro', 'gravity']):
                    if len(sensor_tmp[j]) > 0:
                        d[k] = smooth(sensor_tmp[j])
                
                path.append(d)
                sensor_tmp = [[], [], [], [], []]
                google_tmp = None
                gps_tmp = None
                wifi_tmp = None             
                last_ts = ts
            
        paths.append(path)
        
    return paths

def align_grid(sim, locs1, locs2):
    tmp = np.zeros(len(locs2))
    for loc,s in zip(locs1, sim):
        i = get_closest_grid(loc, locs2)
        tmp[i] = s
    return tmp
    

def load_mag_model(path):
    mag_fp = read_magnetic_fingerprint(os.path.join(path, 'mag_fingerprint.txt'))
    mag = MagDTWMatching(mag_fp)
    mag_w = np.loadtxt(os.path.join(path, 'mag_weight.txt'))
    return mag, mag_w

def load_wifi_model(path):
    wifi_w = np.loadtxt(os.path.join(path, 'wifi_weight.txt'))
    wifi = tf.saved_model.load(path)
    return wifi, wifi_w


def load_preprocessor(path):
    if os.path.exists(os.path.join(path, 'preprocessor')):
        with open(os.path.join(path, 'preprocessor'), 'rb') as f:
            preprocessor = pickle.load(f)
        return preprocessor



def location_sim(signal, site, rp_locs):
    g = lambda x,loc,accuracy: np.exp(-((x[0]-loc[0])**2 + (x[1]-loc[1])**2) / (2*(accuracy * accuracy / 2.27)))
    coord = [signal[0], signal[1], 1]
    loc = np.matmul(coord, site.transformation)
    return [g(x, loc, signal[2]*site.scale) for x in rp_locs], loc

In [9]:
#preprocessor:


In [16]:
gridsize = 20
site = LTA()
map_constraints = site.in_constraints, site.out_constraints
map_path = '../map/HKUST/HKUSTMap1F.jpg'
preprocessor = load_preprocessor(os.path.join(site.log_dir, 'wifi2', 'preprocessor'))
mag, mag_w = load_mag_model(os.path.join(site.log_dir, 'mag'))
wifi, wifi_w = load_wifi_model(os.path.join(site.log_dir, 'wifi2'))
rp_locs, rp_features = read_reference_points(os.path.join('..', site.model_dir, 'wifi2', 'grid.txt'))
out_locs = []#generate_filled_grid([([2070, 2400, 2400, 2070], [1900, 1900, 2300, 2300])], gridsize)
# area_locs = rp_locs + out_locs
area_locs = rp_locs
wifi_w = np.hstack((wifi_w, np.ones(len(out_locs))))
mag_w = np.hstack((mag_w, np.ones(len(out_locs))))
rps = wifi.predict(rp_features)

PermissionError: [Errno 13] Permission denied: '.'

In [13]:
idx = 0
data = readData(os.path.join('..', site.dataset_dir))
ins_data = readInsData(os.path.join('..', site.dataset_dir))
len(data)

0

In [17]:
%matplotlib widget

N = 1000
pf = ParticleFilter(N, area_locs, map_constraints)
pdr = PDR()
fig = plt.figure()
im = plt.imread(map_path)
plt.imshow(im)
# plt.xlim(4000, 6000)
# plt.ylim(2200,1500)
scat = plt.scatter(pf.particles[:,0], pf.particles[:,1], s=3)
curr = plt.scatter(0,0, s=5, c='r')
truth = plt.scatter(0,0, s=5, c='lightgreen')
plt.show()

NameError: name 'area_locs' is not defined

In [7]:
lastLoc = data[idx][0]['loc']
mag_seq, mag_tmp = [], []
i = 0

In [8]:
d = data[idx][i]
loc = d['loc']

dl = pdr.get_travel_distance(d['acc'])
dh = pdr.get_heading_change(d['heading'])
print(d['timestamp'], dh, pdr.total_step)

pf.predict(dh, dl)
scat.set_offsets(pf.particles[:,:2])

sim = np.ones(len(area_locs))
if 'wifi' in d:
    print('wifi')
    wifi_feature = preprocessor.preprocess(*d['wifi']).astype(np.float32)
    wifi_sim = np.exp(-tf.norm(rps - wifi.predict(wifi_feature), axis=1) ** 2/50)
    wifi_sim = align_grid(wifi_sim, rp_locs, area_locs)
    sim *= wifi_sim
#     wifi_sim = wifi.compute_similarity(tf.constant(rp_features), tf.cast(tf.tile(wifi_feature, [len(rp_features),1]), tf.float32)).numpy()

if 'mag' in d:
    print('mag')
    mag_seq = (mag_seq + np.linalg.norm(d['mag'], axis=1).tolist())[-100:]
    mag_sim = mag.predict(mag_seq, rp_locs, lastLoc)
    mag_sim = align_grid(mag_sim, rp_locs, area_locs)
    sim *= mag_sim

if 'google' in d:
    print('google')
    google_sim, _ = location_sim(d['google'], site, area_locs)
    sim *= google_sim
    
if 'gps' in d:
    print('gps')
    gps_sim, _ = location_sim(d['gps'], site, area_locs)
    sim *= gps_sim


pf.update(sim)
lastLoc = pf.estimate()
print(pf.neff(), lastLoc, loc, distance(lastLoc, loc)*3/40)
curr.set_offsets(lastLoc)
truth.set_offsets(loc)

if pf.neff() < N / 2:
    pf.resample()

i += 1

1572524933104 4.685898034298122 1
wifi
mag
google
96.17325858316872 [4404.21269359 1722.49475288] (4408.559739743152, 1677.7304539520826) 3.3731155309911345


In [8]:
idx = 0
ttt = 999
p = 1
sigma_w = 5
sigma_m = 1e4
distanceRange = 120

In [ ]:
N=1000
pf = ParticleFilter(N, area_locs, map_constraints)
pdr = PDR()
results_sifu = np.empty((0,4))
results_time = []
timestamps1 = []
lastLoc = data[idx][0]['loc']
mag_seq = []


for i in range(len(data[idx])):
    d = data[idx][i]
    loc = d['loc']
    start_time = time.time()
    
    dl = pdr.get_travel_distance(d['acc'])
    dh = pdr.get_heading_change(d['heading'])
#     print(ts, dl, dh, pdr.total_step)
    pf.predict(dh, dl)
    scat.set_offsets(pf.particles[:,:2])
    
    sim = np.ones(len(area_locs))
    if 'wifi' in d and i < ttt and i % p == 0:
#         print('wifi')
        wifi_feature = preprocessor.preprocess(*d['wifi']).astype(np.float32)
        wifi_sim = np.exp(-tf.norm(rps - wifi.predict(wifi_feature), axis=1) ** 2/(2*sigma_w*sigma_w))
        wifi_sim = np.hstack((wifi_sim, np.zeros(len(out_locs))))
#         wifi_sim = align_grid(wifi_sim, rp_locs, area_locs)
        sim *= wifi_sim ** wifi_w     

    if 'mag' in d and i < ttt:
        #         print('mag')
        mag_seq = (mag_seq + np.linalg.norm(d['mag'], axis=1).tolist())[-100:]
        mag_sim = mag.predict(mag_seq, area_locs, lastLoc, sigma=sigma_m, distanceRange=distanceRange)
#         mag_sim = align_grid(mag_sim, rp_locs, area_locs)
        sim *= mag_sim  ** mag_w

    if 'google' in d:
#         print('google')
        google_sim, g_loc = location_sim(d['google'], site, area_locs)   
#         print(g_loc)
        sim *= google_sim
        
    if 'gps' in d:
#         print('gps')
        gps_sim, _ = location_sim(d['gps'], site, area_locs)
        sim *= gps_sim
    
    pf.update(sim) 
    lastLoc = pf.estimate()
    print(lastLoc, loc, distance(lastLoc, loc)*3/40)

    results_time.append(time.time() - start_time)  
    timestamps1.append(d['timestamp'])
    results_sifu = np.vstack([results_sifu, [lastLoc[0], lastLoc[1], loc[0], loc[1]]])    
    curr.set_offsets(lastLoc)
    truth.set_offsets(loc)
    
    if pf.neff() < N/2:
#         print(pf.neff())
        pf.resample()


In [82]:
N=1000
pf = ParticleFilter(N, area_locs, map_constraints)
pdr = PDR()
results_naive = np.empty((0,4))
results_wifi = np.empty((0,4))
results_fsw = np.empty((0,4))
results_google = np.empty((0,4))
timestamps = []
lastLoc = data[idx][0]['loc']
g_loc = None
mag_seq = []

for i in range(len(data[idx])):
    d = data[idx][i]
    loc = d['loc']
    start_time = time.time()
    
    dl = pdr.get_travel_distance(d['acc'])
    dh = pdr.get_heading_change(d['heading'])
#     print(ts, dl, dh, pdr.total_step)
    pf.predict(dh, dl)
    scat.set_offsets(pf.particles[:,:2])
    
    sim = np.ones(len(area_locs))
    if 'wifi' in d and i < ttt and i % p == 0:
#         print('wifi')
        wifi_feature = preprocessor.preprocess(*d['wifi']).astype(np.float32)
        wifi_sim = np.exp(-tf.norm(rps - wifi.predict(wifi_feature), axis=1) ** 2/(2*sigma_w*sigma_w))
        wifi_sim = np.hstack((wifi_sim, np.zeros(len(out_locs))))
#         wifi_sim = align_grid(wifi_sim, rp_locs, area_locs)
        sim *= wifi_sim        

    if 'mag' in d and i < ttt:
        #         print('mag')
        mag_seq = (mag_seq + np.linalg.norm(d['mag'], axis=1).tolist())[-100:]
        mag_sim = mag.predict(mag_seq, area_locs, lastLoc, sigma=sigma_m, distanceRange=distanceRange)
#         mag_sim = align_grid(mag_sim, rp_locs, area_locs)
        sim *= mag_sim

    if 'google' in d:
#         print('google')
        google_sim, g_loc = location_sim(d['google'], site, area_locs)   
#         print(g_loc)
        sim *= google_sim
        
    if 'gps' in d:
#         print('gps')
        gps_sim, _ = location_sim(d['gps'], site, area_locs)
        sim *= gps_sim
    
    pf.update(sim) 
    lastLoc = pf.estimate()
#     print(lastLoc, loc, distance(lastLoc, loc)*3/40)
    
    results_naive = np.vstack([results_naive, [lastLoc[0], lastLoc[1], loc[0], loc[1]]])    
    curr.set_offsets(lastLoc)
    truth.set_offsets(loc)
    
    if i < ttt:
        pred2 = rp_locs[wifi_sim.argmax()]
        results_wifi = np.vstack([results_wifi, [pred2[0], pred2[1], loc[0], loc[1]]]) 
        pred2 = rp_locs[(mag_sim*wifi_sim).argmax()]
        results_fsw = np.vstack([results_fsw, [pred2[0], pred2[1], loc[0], loc[1]]]) 
        timestamps.append(d['timestamp'])
    if g_loc is not None:
        results_google = np.vstack([results_google, [g_loc[0], g_loc[1], loc[0], loc[1]]])
    else:
        results_google = np.vstack([results_google, [-1, -1, loc[0], loc[1]]])

    if pf.neff() < N/2:
#         print(pf.neff())
        pf.resample()


In [84]:
x1 = list(sorted([distance(xx[0:2], xx[2:4])*3/40 for xx in results_sifu[:]]))
y1 = np.cumsum(x1) / sum(x1)
x2 = list(sorted([distance(xx[0:2], xx[2:4])*3/40 for xx in results_wifi[:]]))
y2 = np.cumsum(x2) / sum(x2)
x3 = list(sorted([distance(xx[0:2], xx[2:4])*3/40 for xx in results_fsw[:]]))
y3 = np.cumsum(x3) / sum(x3)
x4 = list(sorted([distance(xx[0:2], xx[2:4])*3/40 for xx in results_naive[:]]))
y4 = np.cumsum(x4) / sum(x4)
x5 = list(sorted([distance(xx[0:2], xx[2:4])*3/40 for xx in results_google]))
y5 = np.cumsum(x5) / sum(x5)

print(np.mean(x1), np.mean(x2), np.mean(x3), np.mean(x4), np.mean(x5), np.mean(results_time))
%matplotlib widget
fig, ax = plt.subplots()
plt.plot(x1,y1, 'k-', label='SiFu')
plt.plot(x2,y2, 'r--', label='WiDeep')
plt.plot(x3,y3, 'g-.', label='FSW')
plt.plot(x4,y4, 'b:', label='Naive')
plt.plot(x5,y5, 'c:', label='Google')
plt.grid(b=True, which='major', color='#dddddd', linestyle='-')

handles,labels = ax.get_legend_handles_labels()
handles = [handles[0], handles[3], handles[2], handles[1], handles[4]]
labels = [labels[0], labels[3], labels[2], labels[1], labels[4]]

plt.ylabel('Cumulative Probability')
plt.xlabel('Localization Error (m)')
plt.xlim(0,12)
plt.ylim(0,1)
plt.legend(handles, labels)
plt.show()

4.45856394817217 7.643980243371604 6.489896217710062 5.053522512608893 7.2389065268240955 0.6218734293273


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
times = (np.array(timestamps) - timestamps[0]) / 1000
times1 = (np.array(timestamps1) - timestamps[0]) / 1000

plt.figure(figsize=(10,5))
plt.plot(times1, [distance(xx[0:2], xx[2:4])*3/40 for xx in results_sifu[:]], 'k-', label='SiFu')
plt.plot(times, [distance(xx[0:2], xx[2:4])*3/40 for xx in results_wifi[:]], 'r--', label='WiDeep')
plt.plot(times, [distance(xx[0:2], xx[2:4])*3/40 for xx in results_fsw[:]], 'g-.', label='FSW')
plt.plot(times1, [distance(xx[0:2], xx[2:4])*3/40 for xx in results_naive[:]], 'b:', label='Naive')
plt.plot(times1, [distance(xx[0:2], xx[2:4])*3/40 for xx in results_google[:]], 'c:', label='Google')
plt.grid(b=True, which='major', color='#dddddd', linestyle='-')

handles,labels = ax.get_legend_handles_labels()
handles = [handles[4], handles[1], handles[2], handles[3], handles[0]]
labels = [labels[4], labels[1], labels[2], labels[3], labels[0]]

plt.ylabel('Localization Error (m)')
plt.xlabel('Time (s)')
plt.xlim(0, max(times1))
plt.ylim(0)
plt.legend(handles, labels)
plt.show()

NameError: name 'timestamps' is not defined

In [13]:
%matplotlib widget

x1,x2,y1,y2 = 2810,3700,1350,1600
x1,x2,y1,y2 = 4200,5700,1500,2100
# x1,x2,y1,y2 = 1800, 2800, 1000, 2500

# plt.figure(figsize=(20,10))
im = plt.imread(map_path)[y1:y2,x1:x2]
plt.imshow(im)
plt.plot(results_sifu[:,2]-x1, results_sifu[:,3]-y1, lw=1)
plt.plot(results_sifu[:,0]-x1, results_sifu[:,1]-y1, lw=1)

plt.show()
plt.savefig('../../results/test.jpg', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [203]:
fig, ax = plt.subplots()
im = plt.imread(map_path)
ax.imshow(im)
heatmap = np.zeros(im.shape[:2])
s = sim
for ss, loc in zip(s, rp_locs):
    x, y = loc
    heatmap[int(y-gridsize/2):int(y+gridsize/2), int(x-gridsize/2):int(x+gridsize/2)] = ss

heatmap = np.ma.masked_where(heatmap == 0, heatmap)
ax.imshow(heatmap)
fig.show()

print(max(s), min(s))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.7990204165779999 0.0


In [ ]:
for a,b in zip(sim, rp_locs):
    if a > 0.5:
        print(a,b)

In [6]:
# %matplotlib inline
%matplotlib widget

pdr = PDR()
# acc = [np.mean(d['acc'][i*len(d['acc'])//2: (i+1)*len(d['acc'])//2],0) for _,_,d in data[idx] for i in range(2)]
acc = [x for d in data[idx] for x in d['acc']]
# acc = [[-1.1707647, 3.6750996, 8.485051], [-0.48602295, 3.9336734, 9.816227], [-0.35673606, 1.3934253, 5.707777], [-1.3335704, 2.9999347, 8.774749], [-7.7787614, 10.702082, -5.777209], [-0.260968, 3.6607344, 8.568848], [-0.533907, 4.4244847, 8.166622], [-0.82360536, 2.1452048, 10.840945], [-1.6998832, 2.9592333, 8.748413], [-1.3239936, 3.0597897, 7.8290396], [-0.05746084, 3.5122938, 6.8929067], [0.02633622, 5.0661306, 11.092337], [-1.2354081, 1.7741034, 10.007763], [-0.5243302, 3.2130187, 11.389217], [-0.32321724, 4.402937, 9.466674], [0.9840169, 3.2393548, 11.152191], [-0.50038815, 3.359065, 2.6312277], [-0.39983168, 4.4723687, 7.1490865], [-0.9959879, 5.08289, 10.04607], [-2.3175871, 1.8698715, 10.635044], [-0.08379706, 3.8618474, 5.985504], [-0.7757214, 4.644751, 7.45315], [-1.1013328, 5.12838, 10.730812], [-0.5482722, 2.2050598, 8.738836]]
# acc = np.linalg.norm(acc, axis=1)
acc = pdr._butter_lowpass_filter(acc, 2,20,2)
# acc = pdr._low_pass_filter(acc)
acc = acc[0:1000, 2]
plt.figure(figsize=(10,5))
plt.plot(np.arange(0,len(acc)), acc)
print(pdr._count_step(acc,H=9.5,W=5,D=20))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(27, 969)


In [7]:
%matplotlib widget

idx = 0
pdr = PDR()
locs = []
truths = []
# x, y, heading = 4405,1680, np.pi
x, y = data[idx][0]['loc']
heading = np.pi *0.99
for d in data[idx]:
    dl = pdr.get_travel_distance(d['acc'])
    dh = pdr.get_heading_change(d['heading'])
    heading += dh
    
#     speed = pdr.get_speed(d['acc'], d['mag'], d['gravity'], d['timestamp'])
#     print(d['loc'], speed)
           
    x += np.cos(heading) * dl
    y += np.sin(heading) * dl
    
    locs.append((x,y))
    truths.append(d['loc'])

im = plt.imread(map_path)
plt.imshow(im)
plt.scatter(*zip(*locs), s=1)
plt.scatter(*zip(*truths), c='r', s=1)
plt.show()

0.5316555699225248
0.5035577877261131
0.5025045709361526
0.5418907012275239
0.596271315319443
0.5813091707383417
0.5629284342018848
0.5418951340843262
0.5461524409301379
0.5461524409301379
0.510916086483273
0.5125310733415674
0.5157164574989275
0.525094565256678
0.5570549799718314
0.5717635140846608
0.5717635140846608
0.5748344587651724
0.5564894143023602
0.5674045699248275
0.5616567788625055
0.5453148600473636
0.5499323573998284
0.5499323573998284
0.5324749254104486
0.510464157765357
0.5952765403254276
0.5994304488860831
0.5904058556095457
0.5832719888769778
0.5807243535712534
0.5756560347796084
0.5766689932083053
0.5771696734806607
0.5771696734806607
0.6187342082576639
0.6255927519624582
0.6171108856650345
0.6171108856650345
0.5958882359341008
0.6042606172050363
0.6260762256400731
0.6252224327879774
0.5957972190340001
0.6475618376588582
0.6366062200849184
0.6011747249521471
0.60817928297189
0.6363420472867124
0.6367585421602318
0.612578486763009
0.6099356220126846
0.6323942504787277


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
pdr.total_step

59

In [1]:
%matplotlib widget

im = plt.imread(map_path)
plt.imshow(im)

for x1,y1,x2,y2 in site.in_constraints:
    plt.plot([x1,x2],[y1,y2], ms=10,c='r')
for out_constraint in site.out_constraints:
    for x1,y1,x2,y2 in out_constraint:
        plt.plot([x1,x2],[y1,y2], ms=10,c='b')
plt.show()

NameError: name 'plt' is not defined

In [63]:
%matplotlib widget

idx = 0
signals = [[],[],[],[],[]]
signal_lookup = {1:0,2:1,3:2,4:3,9:4}
last_ts = ins_data[idx][0]['timestamp']
last_h = -1

pdr = PDR()
locs = []
# x, y, heading = 4405,1680, np.pi
x, y, heading = 3620,1480, np.pi*0.9

for s in ins_data[idx]:
    if s['type'] in {1,2,4,9}:
        signals[signal_lookup[s['type']]].append(s)
    
    if s['timestamp'] - last_ts >= 400:
        dl = pdr.get_travel_distance(smooth2(signals[0]))
        seq_orientation = turning_detection(*signals)
        
        dh = np.mean(seq_orientation,0)[0] - last_h if last_h != -1 else np.mean(seq_orientation,0)[0]
        last_h = np.mean(seq_orientation,0)[0]
        heading += dh
   
        x += np.cos(heading) * dl
        y += np.sin(heading) * dl
    
        locs.append((x,y))
        
        signals = [[],[],[],[],[]]
        last_ts = s['timestamp']

im = plt.imread(map_path)
plt.imshow(im)
plt.scatter(*zip(*locs), s=1)
plt.show()

NameError: name 'smooth2' is not defined

In [316]:
seq_acc = [x for x in ins_data[idx] if x['type'] == 1]
seq_mag = [x for x in ins_data[idx] if x['type'] == 2]
seq_gyro = [x for x in ins_data[idx] if x['type'] == 4]
seq_gravity = [x for x in ins_data[idx] if x['type'] == 9]

In [4]:


np.cos

NameError: name 'np' is not defined

In [272]:
idx = 1
ttt = 65
p = 1
sigma_w = 5
sigma_m = 1e4
distanceRange = 120

In [277]:
N=1000
pf = ParticleFilter(N, area_locs, map_constraints)
pdr = PDR()
results_sifu = np.empty((0,4))
results_euclidean = np.empty((0,4))
results_cosine = np.empty((0,4))
results_time = []
timestamps1 = []
lastLoc = data[idx][0]['loc']
mag_seq = []


for i in range(len(data[idx])):
    d = data[idx][i]
    loc = d['loc']
    
    if i < ttt and 'wifi' in d:
        wifi_feature = preprocessor.preprocess(*d['wifi']).astype(np.float32)
        wifi_sim = np.exp(-tf.norm(rps - wifi.predict(wifi_feature), axis=1) ** 2/(2*sigma_w*sigma_w))
        pred2 = [0,0]
#         for l, w in zip(rp_locs, wifi_sim):
        ws = 0
        for ti in np.argpartition(wifi_sim, -30)[-30:]:
            l = rp_locs[ti]
            w = wifi_sim[ti]
            pred2[0] += l[0] * w
            pred2[1] += l[1] * w
            ws += w
        pred2[0] /= ws
        pred2[1] /= ws
#         pred2 = rp_locs[wifi_sim.argmax()]
        results_sifu = np.vstack([results_sifu, [pred2[0], pred2[1], loc[0], loc[1]]]) 
    
        wifi_sim = np.exp(-tf.norm(np.array(rp_features) - wifi_feature, axis=1) ** 2/10)
        pred2 = rp_locs[wifi_sim.argmax()]
        results_euclidean = np.vstack([results_euclidean, [pred2[0], pred2[1], loc[0], loc[1]]])
        
        wifi_sim = ((np.dot(np.array(rp_features), wifi_feature.T)).ravel()/(np.linalg.norm(rp_features,axis=1)*np.linalg.norm(wifi_feature))).ravel()
        pred2 = rp_locs[wifi_sim.argmax()]
        results_cosine = np.vstack([results_cosine, [pred2[0], pred2[1], loc[0], loc[1]]]) 



In [278]:
(np.mean([distance(xx[0:2], xx[2:4])*3/40 for xx in results_sifu[:]]),
 np.mean([distance(xx[0:2], xx[2:4])*3/40 for xx in results_euclidean[:]]),
 np.mean([distance(xx[0:2], xx[2:4])*3/40 for xx in results_cosine[:]]),
)

(4.379547367360435, 6.117701488389211, 5.565175218878785)